In [11]:
import sys         ###mcl4header
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal  #追加

In [12]:
class Particle:
    def __init__(self, init_pose):
        self.pose = init_pose


In [13]:
class Mcl: ###Mcl4（draw以下省略）###
    def __init__(self, init_pose, num, motion_noise_stds):   #引数追加
        self.particles = [Particle(init_pose) for i in range(num)]
        
        v = motion_noise_stds #5-7行目追加
        c = np.diag([v["nn"]**2, v["no"]**2, v["on"]**2, v["oo"]**2])
        self.motion_noise_rate_pdf = multivariate_normal(cov=c)
        
    def motion_update(self, nu, omega, time): #追加
        print(self.motion_noise_rate_pdf.cov_object)
        
    def draw(self, ax, elems): 
        xs = [p.pose[0] for p in self.particles]
        ys = [p.pose[1] for p in self.particles]
        vxs = [math.cos(p.pose[2]) for p in self.particles] 
        vys = [math.sin(p.pose[2]) for p in self.particles] 
        elems.append(ax.quiver(xs, ys, vxs, vys, color="blue", alpha=0.5))

In [14]:
class EstimationAgent(Agent):   ###EstimationAgent4###
    def __init__(self, time_interval, nu, omega, estimator): #time_intervalを追加
        super().__init__(nu, omega)
        self.estimator = estimator
        self.time_interval = time_interval #追加
        
    def draw(self, ax, elems):
        self.estimator.draw(ax, elems)

In [15]:
initial_pose = np.array([0, 0, 0]).T   ###mcl_test4（下の行列出力も掲載）###
estimator = Mcl(initial_pose, 100, motion_noise_stds={"nn":0.01, "no":0.02, "on":0.03, "oo":0.04})
a = EstimationAgent(0.1, 0.2, 10.0/180*math.pi, estimator)
estimator.motion_update(0.2, 10.0/180*math.pi, 0.1)